# Assignment 3_Titanic_Yiyan_Wan

In [1]:
# General
import numpy as np 
import pandas as pd 

# data visualization
import seaborn as sns
%matplotlib inline
from matplotlib import pyplot as plt
from matplotlib import style

# Algorithms
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.naive_bayes import GaussianNB
import sklearn.model_selection as cv
import random



C:\Users\wanyi\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
df_train = pd.read_csv("../Titanic Survival/train.csv")
df_test = pd.read_csv("../Titanic Survival/holdout_test.csv")

FileNotFoundError: [Errno 2] File ../Titanic Survival/train.csv does not exist: '../Titanic Survival/train.csv'

In [ ]:
def cal_missing_val(df):
    data_dict = {}
    for col in df.columns:
        data_dict[col] = (df[col].isnull().sum()/df.shape[0])*100
    return pd.DataFrame.from_dict(data_dict, orient='index', columns=['MissingValueInPercentage'])

print(cal_missing_val(df_train))
print(cal_missing_val(df_test))


In [ ]:
print(df_train.info())
print(df_test.info())

In [ ]:
survived = 'survived'
not_survived = 'not survived'
fig, axes = plt.subplots(nrows=1, ncols=2,figsize=(10, 4))

women = df_train[df_train['Sex']=='female']
ax = sns.distplot(women[women['Survived']==1].Age.dropna(), bins=18, label = survived, ax = axes[0], kde =False)
ax = sns.distplot(women[women['Survived']==0].Age.dropna(), bins=40, label = not_survived, ax = axes[0], kde =False)
ax.legend()
ax.set_title('Female')


men = df_train[df_train['Sex']=='male']
ax = sns.distplot(men[men['Survived']==1].Age.dropna(), bins=18, label = survived, ax = axes[1], kde = False)
ax = sns.distplot(men[men['Survived']==0].Age.dropna(), bins=40, label = not_survived, ax = axes[1], kde = False)
ax.legend()
ax.set_title('Male')

In [ ]:
# Conclusion: 
# Male/Femal infants had higher probability to survive than adults
# Male more likely to survive in the age of 20 - 40 
# Female more likely to survice in the age of 20 - 40

In [ ]:
FacetGrid = sns.FacetGrid(df_train, row='Embarked', size=4.5, aspect=1.6)
FacetGrid.map(sns.pointplot, 'Pclass', 'Survived', 'Sex', palette=None,  order=None, hue_order=None )
FacetGrid.add_legend()

In [ ]:
# Conclution: male more likely (than woman) to survive if he was embarked from C and woman more likely (than man) to survive if she embarked from Q and S.  

In [ ]:
sns.barplot(x='Pclass', y='Survived', data = df_train)

In [ ]:
# Conclution: higher social class had higher surviving rate

# 1. Proccesing "SibSp" and "Parch"
* combining the features to show whether this traveller is alone or with some company 


In [ ]:
df_train["family"] = df_train['SibSp'] + df_train['Parch']
df_train.loc[df_train['family']> 0,"not_alone"] = 1
df_train.loc[df_train['family'] == 0, "not_alone"]  = 0
df_train['not_alone'] = df_train['not_alone'].astype(int)


In [ ]:
df_test["family"] = df_test['SibSp'] + df_test['Parch']
df_test.loc[df_test['family']> 0,"not_alone"] = 1
df_test.loc[df_test['family'] == 0, "not_alone"]  = 0
df_test['not_alone'] = df_test['not_alone'].astype(int)

# 2. Imputting "Embarked" by fillna by random

In [ ]:
# Rationale: Only 2 out of 891 missing value (low %) and port embark is not that relevant to survival 
df_train['Embarked'].fillna(random.choice(['c','q','s']), inplace=True) 
df_test['Embarked'].fillna(random.choice(['c','q','s']), inplace=True) 


# 3. Imputting "Age" by fillna with random normal distribution based on age mean and std.

In [ ]:
mean = df_train["Age"].mean()
std = df_train["Age"].std()
is_null = df_train["Age"].isnull().sum()
random_age = np.random.randint(mean - std, mean + std, size = is_null)
age_slice = df_train["Age"].copy()
age_slice[np.isnan(age_slice)] = random_age
df_train["Age"] = age_slice
df_train["Age"] = df_train["Age"].astype(int)

In [ ]:
mean = df_test["Age"].mean()
std = df_test["Age"].std()
is_null = df_test["Age"].isnull().sum()
random_age = np.random.randint(mean - std, mean + std, size = is_null)
age_slice = df_test["Age"].copy()
age_slice[np.isnan(age_slice)] = random_age
df_test["Age"] = age_slice
df_test["Age"] = df_test["Age"].astype(int)

# 4. Fare converting to int64

In [ ]:
df_train['Fare'] = df_train['Fare'].fillna(0)
df_train['Fare'] = df_train['Fare'].astype(int)

df_test['Fare'] = df_test['Fare'].fillna(0)
df_test['Fare'] = df_test['Fare'].astype(int)

# 5. One-hot-encoding for 'Pclass'

In [ ]:
df_train = pd.concat([df_train,pd.get_dummies(df_train['Pclass'])],axis =1)
df_test = pd.concat([df_test,pd.get_dummies(df_test['Pclass'])],axis =1)

In [ ]:
df_train = df_train.rename(columns = {1:'Class1',2:'Class2',3:'Class3' })
df_test = df_test.rename(columns = {1:'Class1',2:'Class2',3:'Class3' })

# 6. Label-encoding sex 

In [ ]:
genders = {"male": 0, "female": 1}
df_test['Sex'] = df_test['Sex'].map(genders)
df_train['Sex'] = df_train['Sex'].map(genders)

# 7. One-hot-encoding for 'Embarked'

In [ ]:
df_train = pd.get_dummies(df_train, columns =['Embarked']) 
df_test = pd.get_dummies(df_test, columns =['Embarked']) 

# 8. Drop useless columns 

* dropping the following columns

    * 'family','SibSp','Parch' have been converted for not_alone columns 
    * 'Name' and 'Ticket' and 'PassengerId' have no value on the determination of survivial
    * 'Cabin' may have some values but it was highly missing and Pclass would provide the some information related to the social status.  
    * 'Pclass' has been imputed

In [ ]:
print(df_train.columns)
df_test.columns

In [ ]:
df_train = df_train.drop(['Ticket','Parch','SibSp','Cabin','family','Name','Pclass','PassengerId'], axis=1)
df_test = df_test.drop(['Ticket','Parch','SibSp','Cabin','family','Name','Pclass','PassengerId'], axis=1)

# 11. Final look at the train and test dataset

In [ ]:
df_train.head()

In [ ]:
df_test.head()

# 12. Splitting train data into train and validation before applying model to the test data

In [ ]:
y=df_train['Survived']
cols=['Sex','Age','Fare','Embarked_C','Embarked_Q','Embarked_S','not_alone','Class1','Class2','Class3']
x=df_train[cols]
(x_train, x_test, y_train, y_test) = cv.train_test_split(x, y, test_size=.20)

# 13. Applying Model_1: Decision Tree 

In [26]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier()
dt.fit(x_train, y_train)
y_pred_dt = dt.predict(x_test)

# Test set F1 score

In [27]:
from sklearn.metrics import f1_score
y_pred_dt = dt.predict(x_test)
test_set_F1_dt = f1_score(y_test, y_pred_dt,average = 'weighted')
print("Test set F1 score of dt: {}".format(round(test_set_F1_dt*100),2))

Test set F1 score of dt: 75.0


# CV set F1 score

In [28]:
from sklearn.model_selection import cross_val_score
# Compute the array containing the 10-folds CV MSEs
CV_set_F1_dt = cross_val_score(dt, x_train, y_train, cv=10, scoring='f1_weighted',n_jobs=-1).mean()
print('CV F1 score: {}'.format(round(CV_set_F1_dt*100,2)))

CV F1 score: 77.54


# Train set F1 score

In [29]:
dt.fit(x_train, y_train)
y_pred_dt = dt.predict(x_train)
train_set_F1_dt = f1_score(y_train, y_pred_dt,average = 'weighted')
print('Train F1 score: {}'.format(round(train_set_F1_dt*100,2))) 

Train F1 score: 96.88


* Since CV score < Train score or Test score, so overfitting ---> Tune parameters by decreasing complexity by decreasing max depth and increasing min samples per leaf  

# Tuning hyperparameters until CV F1 score  ~ Train F1 score

In [30]:
dt = DecisionTreeClassifier(max_depth=3, min_samples_leaf=4, random_state=0)
dt.fit(x_train, y_train)
y_pred_dt = dt.predict(x_test)

In [31]:
from sklearn.model_selection import cross_val_score
CV_set_F1_dt = cross_val_score(dt, x_train, y_train, cv=10, 
                                  scoring='f1_weighted', 
                                  n_jobs=-1).mean()
print('CV F1 score: {}'.format(round(CV_set_F1_dt*100,2)))

CV F1 score: 80.71


In [32]:
dt.fit(x_train, y_train)
y_pred_dt = dt.predict(x_train)
train_set_F1_dt = f1_score(y_train, y_pred_dt,average = 'weighted')
print('Train F1 score: {}'.format(round(train_set_F1_dt*100,2))) 

Train F1 score: 82.66


In [33]:
from sklearn.metrics import f1_score
y_pred_dt = dt.predict(x_test)
test_set_F1_dt = f1_score(y_test, y_pred_dt,average = 'weighted')
print("Test set F1 score of dt: {}".format(round(test_set_F1_dt*100),2))

Test set F1 score of dt: 76.0


In [34]:
# After tunning, the parameters above showcase the model is optimal ie not overfitting (little bit underfitting). 
# Using the F-score should be better due to we have large amount of negative (ie more survival than dead) unbalanced data
round(f1_score(y_test, y_pred_dt, average='weighted')*100,2)

75.86

# 14. Applying Model_2: Random Forest 

# Using RandomizedSearchCV to find optimal hyperparameters to prevent overfitting and underfitting.
 * Or we can use the same way under Decision tree to compare both Test Error, Train error and CV error to tune the hyperparameters. 

In [35]:
from sklearn.model_selection import RandomizedSearchCV
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 2000, num = 10)]
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [36]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(random_state = 42)
rf = RandomForestClassifier()
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 300, cv = 5, verbose=2, random_state=42, n_jobs = -1)
rf_random.fit(x_train, y_train)

Fitting 5 folds for each of 300 candidates, totalling 1500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    8.4s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:   56.2s
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done 632 tasks      | elapsed:  3.4min
[Parallel(n_jobs=-1)]: Done 997 tasks      | elapsed:  5.3min
[Parallel(n_jobs=-1)]: Done 1442 tasks      | elapsed:  7.7min
[Parallel(n_jobs=-1)]: Done 1500 out of 1500 | elapsed:  8.0min finished


RandomizedSearchCV(cv=5, error_score='raise-deprecating',
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
                                                    n_estimators='warn',
                                                    n_jobs=None

In [37]:
rf_random.best_params_

{'n_estimators': 1366,
 'min_samples_split': 2,
 'min_samples_leaf': 1,
 'max_features': 'sqrt',
 'max_depth': 10,
 'bootstrap': True}

90.87

In [39]:
from sklearn.metrics import f1_score
round(f1_score(y_test, y_pred_rf, average='weighted')*100,2)

82.0

# 15. Applying Model_3: SVM

# Tuning 

In [40]:
linear_svc = LinearSVC()
linear_svc.fit(x_train, y_train)
y_pred_svm = linear_svc.predict(x_test)

C:\Users\wanyi\Anaconda3\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [41]:
# Testset f1 score
from sklearn.metrics import f1_score
y_pred_svm = linear_svc.predict(x_test)
test_set_F1_svm = f1_score(y_test, y_pred_svm, average = 'weighted')
print("Test set F1 score of dt: {}".format(round(test_set_F1_svm*100),2))

Test set F1 score of dt: 70.0


In [42]:
# CV f1 score
CV_F1_SVM = cross_val_score(linear_svc, x_train, y_train, cv=10, 
                                  scoring='f1_weighted', 
                                  n_jobs=-1).mean()
print("CV F1 score: {}".format(round(CV_F1_SVM*100,2)))

CV F1 score: 71.54


In [43]:
# Train f1 score
linear_svc.fit(x_train, y_train)
y_pred_svm = linear_svc.predict(x_train)

train_set_F1_dt = f1_score(y_train, y_pred_svm,average = 'weighted')

print('Train F1 score: {}'.format(round(train_set_F1_dt*100,2))) 

C:\Users\wanyi\Anaconda3\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Train F1 score: 78.27


* Since CV F1 score < Train F1 score, so overfitting ---> Tune parameters by decreasing complexity by decreasing max depth and increasing min samples per leaf. We could consider decrease the number of feature variables.However, in this case, we already have limited features. We will decrease complexity as a compensation to elimiate overfitting.

In [44]:
linear_svc = LinearSVC(max_iter = 2, random_state = 1)
linear_svc.fit(x_train, y_train)
y_pred_svm = linear_svc.predict(x_test)

C:\Users\wanyi\Anaconda3\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [45]:
# Train f1 score
linear_svc.fit(x_train, y_train)
y_pred_svm = linear_svc.predict(x_train)
train_set_F1_dt = f1_score(y_train, y_pred_svm,average = 'weighted')
print('Train F1 score: {}'.format(round(train_set_F1_dt*100,2))) 

Train F1 score: 52.67


C:\Users\wanyi\Anaconda3\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [46]:
# Testset f1 score
from sklearn.metrics import f1_score
y_pred_svm = linear_svc.predict(x_test)
test_set_F1_svm = f1_score(y_test, y_pred_svm,average = 'weighted')
print("Test set F1 score of dt: {}".format(round(test_set_F1_svm*100),2))

Test set F1 score of dt: 50.0


In [47]:
# CV f1 score
CV_F1_SVM = cross_val_score(linear_svc, x_train, y_train, cv=55, 
                                  scoring='f1_weighted', 
                                  n_jobs=-1).mean()

print("CV F1 score: {}".format(round(CV_F1_SVM*100,2))) 

CV F1 score: 58.8


In [48]:
from sklearn.metrics import f1_score
round(f1_score(y_test, y_pred_svm, average='weighted')*100,2)

49.77

* After tunning, the above result is the best, we can narrow down the difference between CV score and train score
* The F1 scores seems much worse than decision tree and random forest. As such, abandon this model. 

# 16. Applying Model_4: Logistic regression

In [49]:
from scipy.stats import uniform

logreg = LogisticRegression(solver='saga', tol=1e-2, max_iter=200,random_state=0)
distributions = dict(C=uniform(loc=0, scale=4),penalty=['l2', 'l1'])
logreg_cv = RandomizedSearchCV(logreg, distributions, random_state=0)
search = logreg_cv.fit(x_train, y_train)
search.best_params_

C:\Users\wanyi\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


{'C': 2.195254015709299, 'penalty': 'l1'}

In [50]:
logreg = LogisticRegression(solver='saga', tol=1e-2, max_iter=200,C =2.195254015709299, penalty = 'l1' )
logreg.fit(x_train, y_train)
y_pred_logreg = logreg.predict(x_test)
test_set_F1_LogReg = f1_score(y_test, y_pred_logreg,average = 'weighted')
print("Test set F1 score of LogReg: {}".format(round(test_set_F1_LogReg*100),2))

Test set F1 score of LogReg: 62.0


# 17. Applying GradientBoostingClassifier

In [51]:
from sklearn.ensemble import GradientBoostingClassifier
from scipy.stats import uniform as sp_randFloat
from scipy.stats import randint as sp_randInt 

In [52]:
gb = GradientBoostingClassifier(max_depth=4,n_estimators=200,random_state=2)
parameters = {'learning_rate': sp_randFloat(),
              'subsample'    : sp_randFloat(),
              'n_estimators' : sp_randInt(100, 1000),
              'max_depth'    : sp_randInt(4, 10) }
GB_cv = RandomizedSearchCV(estimator=gb, param_distributions = parameters, cv = 2, n_iter = 10, n_jobs=-1)
search = GB_cv.fit(x_train, y_train)
search.best_params_

C:\Users\wanyi\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


{'learning_rate': 0.00598362105537753,
 'max_depth': 4,
 'n_estimators': 734,
 'subsample': 0.5118197974880978}

In [53]:
gb = GradientBoostingClassifier(max_depth=5,n_estimators=276,random_state=2, learning_rate = 0.08475022579896085, subsample = 0.4387677305543283)
gb.fit(x_train, y_train)
y_pred_gb = gb.predict(x_test)
test_set_F1_LogReg = f1_score(y_test, y_pred_gb,average = 'weighted')
print("Test set F1 score of GB: {}".format(round(test_set_F1_LogReg*100),2))

Test set F1 score of GB: 81.0



# Concluion:
* Based on the F1 score above, Randomforest has the highest score.
* We will apply randomforest with specific parameters to apply to the holdout_data

# Models pro and cons:

* RandomForest - pro as it is simple to understand and interpret and easy to use. It has flexibility to describe non-linear dependencies. No need to standarize or normalize features. On the othe rside, RF introduces further randomization in the training of individual trees, which increase the sample size and cost longer time to process data. 



* Decision Tree - pro as it able to capture non-linear relationships between features and labels. Don't require feature scaling such as normalization. On the other side, DTs are unstable, meaning that a small change in the data can lead to a large change in the structure of the optimal decision tree. They are often relatively inaccurate. It usually causes overfitting. 


* Logist Regression - pro as it is a widely used technique because it is very efficient, does not require too many computational resources, it’s highly interpretable, it doesn’t require input features to be scaled, it doesn’t require any tuning, it’s easy to regularize, and it outputs well-calibrated predicted probabilities. For cons, Logistic Regression can only predict a categorical outcome. It is also an Algorithm that is known for its vulnerability to overfitting.Another disadvantage of it is that we can’t solve non-linear problems with logistic regression since it’s decision surface is linear. 

* SVM - Pros: It works really well with a clear margin of separation. It is effective in high dimensional spaces. For Cons: It doesn't perform well when we have large data set because the required training time is higher.Also it is increasing the dimensions that may not be easy to visualize. 

# Final application to Holdout prediction


In [54]:
cols=['Sex','Age','Fare','Embarked_C','Embarked_Q','Embarked_Q','not_alone','Class1','Class2','Class3']
x_holdout = df_test[cols]

In [55]:
#final_model = GradientBoostingClassifier(max_depth=5,n_estimators=276,random_state=2, learning_rate = 0.08475022579896085, subsample = 0.4387677305543283)
final_model = RandomForestClassifier(n_estimators=1577, criterion = "gini", max_depth = 30, min_samples_split = 5, bootstrap= True, min_samples_leaf = 2, max_features = "sqrt")
final_model.fit(x_train, y_train)
y_pred_holdout = final_model.predict(x_holdout)

In [56]:
result = pd.DataFrame(y_pred_holdout)
result.to_csv('result.csv')